This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'v8N-DBa2zKMxAnwz2Mrx'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests
import json


In [2]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2018-01-01&end_date=2018-01-02&api_key=v8N-DBa2zKMxAnwz2Mrx"
request = requests.get(url)


In [3]:
json_file = request.json()

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
json_dict = dict(json_file)

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

 # 1.Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [132]:
'''Get the Zeiss Meditec stock data for the year of 2017 by specifiying the start_date=2017-01-01 
and end_date=2018-01-01'''
url_2017 = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2018-01-01&api_key=v8N-DBa2zKMxAnwz2Mrx"
# Requesting the data using the requests package
request_2017 = requests.get(url_2017)

In [6]:
#Converting the data into Json file
Zeiss_Meditec_Json_file = request_2017.json()

# 2.Convert the returned JSON object into a Python dictionary.

In [139]:
#Converting the json file into dictionary
Zeiss_Meditec_Dict_file = dict(Zeiss_Meditec_Json_file)

######    The column names and the data are seperate in the file and there is so much redundant information in the dictionary file so we can seperate the columns and data and zip them togather to get more clean data

In [8]:
#Getting column names
column_names = Zeiss_Meditec_Dict_file['dataset']['column_names'][1:]

In [9]:
#Getting stock data
data=Zeiss_Meditec_Dict_file['dataset']['data']

In [10]:
#Creating a data dictionary with date as key and stock information for the day as values 
data_dict = {}
for i in data:
    if i[0] in data_dict:
        data_dict[i[0]].append(i[1:])
    else:
        data_dict[i[0]]=i[1:]

In [11]:
#zipping the column names to the day values
for key in data_dict.keys():
    data_dict[key] = dict(zip(column_names, data_dict[key]))

# 3.Calculate what the highest and lowest opening prices were for the stock in this period.

In [58]:
#Using for loop to creating a list with opening prices for all the days
open_list = []
for key,value in data_dict.items():
    if value['Open'] is not None: #There are 3 instances where opening price was None
        open_list.append(value['Open'])
print('Highest opening price is '+str(max(open_list))) #Highest opening price
print('Lowest opening price is '+str(min(open_list))) #Lowest opening price

Highest opening price is 53.11
Lowest opening price is 34.0


# 4.What was the largest change in any one day (based on High and Low price)?

In [78]:
#Using List comprihension to get the list of difference between high price and low price 
#for each day
Change = [value['High']-value['Low'] for key,value in data_dict.items()] 
print('The largest change in any one day is '+str(round(max(Change),2))) #largest change in 2017

The largest change in any one day is 2.81


# 5.What was the largest change between any two days (based on Closing Price)?


In [87]:
#Calculating the largest change between two day (running difference) by using 
#List comprihension and diff fuction from numpy 
import numpy as np
largest_change = max(np.diff([value['Close'] for key,value in data_dict.items()]))
print('The largest change between any two days (based on Closing Price) is '+ 
      str(round(largest_change,2))) #largest change between any two days

The largest change between any two days (based on Closing Price) is 2.56


# 6.What was the average daily trading volume during this year?

In [99]:
#List Comprihension to get a list of 'Traded Volumes' for each day
daily_vol = [value['Traded Volume'] for key,value in data_dict.items()]

#Calculating Average daily volume for 2017 by dividing sum of daily volumes 
#by length of daily volume
avg_daily_vol = sum(daily_vol)/len(daily_vol)

In [96]:
print("The average daily trading volume during this year was " + str(round(avg_daily_vol,2)))

The average daily trading volume during this year was 89124.34


# 7.(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [134]:
#Creating a median fuction to get the median value from a list
def median(x):
    y = sorted(x)
    n = len(x)
    if n%2==1:
        return y[n//2]
    else:
        return (y[(n//2)-1]+y[((n//2)+1)-1])/2

In [135]:
print("The median trading volume during this year was "+ str(median(daily_vol)))

The median trading volume during this year was 76286.0
